# Pandas Analysis

In [73]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Aggregation and Grouping
Here we will use the Planets dataset, available via the Seaborn package. It gives information on planets that astronomers have discovered around other stars (known as extrasolar planets or exoplanets for short). It can be downloaded with a simple Seaborn command.

In [7]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Earlier, we explored some of the data aggregations available for NumPy ```arrays```. As with a one-dimensional NumPy ```array```, for a Pandas ```Series``` the aggregates return a single value.

In [5]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
print("Series sum:", ser.sum())
print("Series mean:", ser.mean())

Series sum: 2.811925491708157
Series mean: 0.5623850983416314


For a ```DataFrame```, by default the aggregates return results within each column. By specifying the ```axis``` argument, you can instead aggregate within each row.

In [8]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
print("Default aggregation by column for mean:\n", df.mean())
print("Aggregation by row for mean:\n", df.mean(axis="columns")) # could also use axis=1


Default aggregation by column for mean:
 A    0.427590
B    0.551528
dtype: float64
Aggregation by row for mean:
 0    0.696360
1    0.185099
2    0.289643
3    0.770650
4    0.506041
dtype: float64


In addition to standard aggregation functions, Pandas includes a convenience function called ```describe()``` that computes several common aggregates for each column and returns the result. Let's take a look at how it works using the planets dataset and dropping rows with missing values.

In [9]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


This can be a useful way to begin understanding the overall properties of a dataset. For example, we see in the ```year``` column that although exoplanets were discovered as far back as 1989, half of all known expolanets were not discovered until 2010 or after. This is largely thanks to the Kepler mission, which is a space-based telescope specifically designed for finding eclipsing planets around other stars.

To go deeper into the data, however, simple aggregates are often not enough. The next level of data summarization is the ```groupby``` operation, which allows you to quickly and efficiently compute aggregates on subsets of data.

## GroupBy: Split, Apply, Combine
Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called groupby operation. The name "group by" comes from a command in the SQL database language, but it is perhaps more illuminative to think of it in the terms first coined by the ```R``` developer Hadley Wickham: split, apply, combine.

![](figures/03.08-split-apply-combine.png)

This makes clear what the ```groupby``` accomplishes:
- The *split* step involves breaking up and grouping a ```DataFrame``` depending on the value of the specified key.
- The *apply* step involves computing some function, usually an aggregate, transformation, or filtering, within the individual groups.
- The *combine* step merges the results of these operations into an output array.

While this could certainly be done manually using some combination of the masking, aggregation, and merging commands covered earlier, an important realization is that the intermediate splits do not need to be explicitly instantiated. Rather, the ```GroupBy``` can (often) do this in a single pass over the data, updating the sum, mean, count, min, or other aggregate for each group along the way. The power of the ``GroupBy`` is that it abstracts away these steps: the user need not think about how the computation is done under the hood, but rather thinks about the operation as a whole.

As a concrete example, let's take a look at using Pandas for the computation shown in this diagram using the planets dataset.

In [9]:
planets.groupby('method')

Notice that what is returned is not a set of ```DataFrames```, but a ```DataFrameGroupBy``` object. This object is where the magic is: you can think of it as a special view of the ```DataFrame```, which does no actual computation until the aggregation is applied. This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.The ```GroupBy``` object supports column indexing in the same way as a ```DataFrame```, and returns a modified GroupBy object. The ```GroupBy``` object supports column indexing in the same way as a ```DataFrame```, and returns a modified ```GroupBy``` object.

In [12]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

### Aggregate, fileter, transform, apply
The preceding discussion focused on aggregation for the combine operation, but there are more options available. In particular, ```GroupBy``` objects have ```aggregate()```, ```filter()```, ```transform()```, and ```apply()``` methods that efficiently implement a variety of useful operations before combining the grouped data.

In [16]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Aggregation
We're now familiar with ```GroupBy``` aggregations with ```sum()```, ```median()```, etc., but the aggregate() method allows for even more flexibility. It can take a string, a function, or a list thereof, and compute all the aggregates at once.

In [17]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Another useful pattern is to pass a dictionary mapping column names to operations to be applied on that column.

In [18]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### Filtering
A filtering operation allows you to drop data based on the group properties. For example, we might want to keep all groups in which the standard deviation is larger than some critical value (e.g., 4 for column ```data2```).

In [22]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)") # Drops rows where key = A based on std value

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

#### Transformation
While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common example is to center the data by subtracting the group-wise mean.

In [24]:
df.groupby('key').transform(lambda x: x - x.mean()) # A lambda function is a Pythonic way to quickly specify a function in-place

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### Apply
The ```apply()``` method lets you apply an arbitrary function to the group results. The function should take a ```DataFrame```, and return either a Pandas object (e.g., ```DataFrame```, ```Series```) or a scalar; the combine operation will be tailored to the type of output returned.

For example, below is an ```apply()``` that normalizes the first column by the sum of the second.

In [25]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)") # Groupby key and apply summation of data2 column sum for each group

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


### Specifying the split key
In the simple examples presented before, we split the ```DataFrame``` on a single column name. This is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here.

##### A list, series, or index as the grouping keys
The key can be any ```series```, ```list```, or ```index``` with a length matching that of the ```DataFrame```.

In [27]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

#### A dictionary or series mapping index to group
Another method is to provide a ```dictionary``` that maps index values to the group keys.

In [29]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
key,,
consonant,12,19


It is also possible to combine methods to form a multi-index grouping.

In [32]:
df2.groupby(['key', mapping]).mean()

,,data1,data2
key,key,,
A,vowel,1.5,4.0
B,consonant,2.5,3.5
C,consonant,3.5,6.0


Below is a more complex example that combines several Python operations (some we've covered to this point and others that are noted in comments). This shows the power of combining many of the operations we've discussed up to this point when looking at realistic datasets. We immediately gain a coarse understanding of when and how planets have been discovered over the past several decades!

In [39]:
decade = 10 * (planets['year'] // 10) # // is floor division that rounds down to the nearest integer
decade = decade.astype(str) + 's' # Changes the integer decade into a string and adds 's' to the end of each decade: e.g., 1980s
decade.name = 'decade' # Gives the Series a name 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0) # Group planets DataFrame by method and decade Series

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


## Pivot tables
We have seen how the ```GroupBy``` abstraction lets us explore relationships within a dataset. A *pivot table* is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data. The *pivot table* takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summary of the data. A *pivot table* is essentially a multidimensional version of ```GroupBy``` aggregation. That is, you split-apply-combine, but both the split and the combine happen across a two-dimensional grid  rather than a one-dimensional index.

For the examples in this section, we'll use the database of passengers on the Titanic available through the Seaborn package.

In [41]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### Manual pivot using Groupby

It is possible to use ```Groupby``` to perform pivot table operations, but it can be cumbersome. To start learning more about this data, we might begin by grouping according to gender, survival status, or some combination thereof. Let's look at survival rate by gender using ```Groupby``` operations. This immediately gives us some insight: overall, three of every four females on board survived, while only one in five males survived!

In [42]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


This is useful, but we might like to go one step deeper and look at survival by both sex and, say, class. Using the vocabulary of ```GroupBy```, we might proceed using something like this: we group by class and gender, select survival, apply a mean aggregate, combine the resulting groups, and then unstack the hierarchical index to reveal the hidden multidimensionality.

In [44]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


### Pivot using built-in pivot_table

Two-dimensional ```GroupBy``` is common enough that Pandas includes a convenience routine, ```pivot_table```, which succinctly handles this type of multi-dimensional aggregation.

In [45]:
titanic.pivot_table('survived', index='sex', columns='class') # default aggfunc is mean

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [48]:
titanic.pivot_table('survived', index='sex', columns='class', aggfunc='std') # specify a different aggfunc 'std'

class,First,Second,Third
sex,,,
female,0.176716,0.271448,0.501745
male,0.484484,0.365882,0.342694


### Multi-level pivot tables
Just as in the ```GroupBy```, the grouping in pivot tables can be specified with multiple levels, and via a number of options. For example, we might be interested in looking at age as a third dimension. We'll bin the age using the ```pd.cut``` function. Further, we can apply the same strategy when working with the columns as well; let's add info on the fare paid using pd.qcut to automatically compute quantiles.

In [54]:
age = pd.cut(titanic['age'], [0, 18, 80])
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', index=['sex', age], columns=[fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

### Additional pivot table options
The full call signature of the ```pivot_table``` method of ```DataFrames``` is as follows:
```
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```
Two of the options, ```fill_value``` and ```dropna```, have to do with missing data and are fairly straightforward; we will not show examples of them here.

The ```aggfunc``` keyword controls what type of aggregation is applied, which is a mean by default. As in the GroupBy, the aggregation specification can be a string representing one of several common choices (e.g., ```'sum'```, ```'mean'```, ```'count'```, ```'min'```, ```'max'```, etc.) or a function that implements an aggregation (e.g., ```np.sum()```, ```min()```, ```sum()```, etc.). Additionally, it can be specified as a dictionary mapping a column to any of the above desired options.

In [61]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':'sum', 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

In [59]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


At times it's useful to compute totals along each grouping. This can be done via the margins keyword.

In [62]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


## References
https://jakevdp.github.io/PythonDataScienceHandbook/